In [ ]:
# Compare-kreport-taxonomic-profiles.ipynb

# The goal of this Jupyter notebook is to facilitate easy comparisons of the taxonomic 
# profiles of multiple metagenomic samples. These analyses require kraken-report 
# (kreport) format files as inputs, and the analyses can be performed for any 
# taxonomic rank in the kreport files (strain, species, genus, family, etc.). 

# Running the notebook requires the pandas, seaborn, and matplotlib Python libraries.

# The kreport format is standard for many profilers (Kraken, Bracken, Centrifuge, MMSeqs2), and 
# there are conversion scripts available for Metamaps, MetaPhlAn3, and MEGAN in:
# https://github.com/PacificBiosciences/pb-metagenomics-tools/tree/master/pb-metagenomics-scripts
# The PacBio MEGAN-LR workflows also produce kreport output formats:
# https://github.com/PacificBiosciences/pb-metagenomics-tools/

# To run your own analyses, you will need to edit the sample/file path information. 
# You can generate counts at different taxnomic levels using the get_all_dfs() function.
# There are examples of how to do this at the species, genus, and family levels, along with
# how to save the count tables and create/save stacked barplot figures. 

# Daniel Portik
# Bioinformatics Scientist, Pacific Biosciences
# dportik@pacificbiosciences.com
# 05/2021

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
########################################################################
# Enter sample and file information here
########################################################################

# Provide the full path to each kreport to include below. 
# In this example we are using four files. The variable name
# is ideally what you want to call the sample in the analysis.

CID_138274 = '/Users/dportik/Documents/Projects/Datasets/CID_138274.megan-RMA-c2c.kreport.txt'
CID_138390 = '/Users/dportik/Documents/Projects/Datasets/CID_138390.megan-RMA-c2c.kreport.txt'
CID_139369 = '/Users/dportik/Documents/Projects/Datasets/CID_139369.megan-RMA-c2c.kreport.txt'
CID_139445 = '/Users/dportik/Documents/Projects/Datasets/CID_139445.megan-RMA-c2c.kreport.txt'

# Create a dictionary with string labels for the files as keys and the 
# file path variables above as the values. The string labels will be used 
# to label the samples in the resulting tables and figures.

# Follow the format below for your own data.

file_dict = {"CID_138274": CID_138274, 
             "CID_138390": CID_138390, 
             "CID_139369": CID_139369, 
             "CID_139445": CID_139445}

In [ ]:
# The series of functions will create three types of pandas dataframes with taxon 
# counts for a specified taxonomic rank. The first dataframe contains absolute counts of 
# the taxa across all samples. It can be exported as a CSV file, and is also transformed to 
# make the second and third dataframes. These additional dataframes will contain the 
# absolute or normalized counts in a transposed format that is used to make the stacked
# barplot figures. Do not edit these functions!

def get_unique_taxa_at_rank(file_dict, rank):
    """
    Function to obtain all unique taxon names at a given rank 
    for a set of kreport files provided in file_dict. Returns 
    sorted list of unique taxon names.
    
    :param file_dict: dictionary with string label as key and file path as val
    :param rank: desired taxonomic rank (e.g., 'S1', 'S', 'G', 'F', etc.)
    :return: sorted list of unique taxon names at rank
    """
    # initiate empty set for taxon names
    taxon_set = set()
    
    # iterate over the file dictionary
    for key, val in file_dict.items():
        
        # create a pandas dataframe, remove whitespace to clean names column of kreport 
        df = pd.read_csv(val, sep = '\t', header=None, skipinitialspace=True,
                         names = ['proportion', 'cumulative_count', 'level_count', 'rank', 'taxid', 'name'])
        
        # get all unique taxon names at rank, add to set
        taxa = [t for t in (df.loc[df['rank'] == rank, 'name']).unique()]
        taxon_set.update(taxa)
    
    # return sorted list of taxon names
    return sorted(taxon_set)

def make_rank_dataframe(file_dict, rank):
    """
    Function to create a dataframe with taxon counts for 
    all kreport files in file_dict at a given taxonomic rank. 
    Returns pandas dataframe.
    
    :param file_dict: dictionary with string label as key and file path as val
    :param rank: desired taxonomic rank (e.g., 'S1', 'S', 'G', 'F', etc.)
    :return: pandas dataframe of taxon counts for specified rank
    """
    
    # initiate empty dictionary
    taxon_count_dict = {}
    
    # get taxon names to use for this rank
    taxon_list = get_unique_taxa_at_rank(file_dict, rank)
    
    # iterate over the file dictionary
    for key, val in file_dict.items():
        
        # add this sample as a key in the taxon count dictionary
        # value is an empty list that will be filled with taxon counts
        # in the order of the taxon_list
        taxon_count_dict[key] = []
        
        # make temp dataframe, remove whitespace to clean names column of kreport 
        df = pd.read_csv(val, sep = '\t', header=None, skipinitialspace=True, 
                         names = ['proportion', 'cumulative_count', 'level_count', 'rank', 'taxid', 'name'])
        
        # iterate over taxon names 
        for taxon in taxon_list:
            # make a subframe with matched name
            temp = df.loc[df['name'] == taxon]
            # add the cumulative count for the taxon name to the dict list
            # this should only be a single name, taking sum as easy way to get integer
            taxon_count_dict[key].append(temp.loc[df['rank'] == rank, 'cumulative_count'].sum())
    
    # create new dataframe from the dictionaries constructed
    df_taxa = pd.DataFrame.from_dict(taxon_count_dict)
    
    # add in a column with the taxon labels
    df_taxa.insert(0, 'Taxa', taxon_list)
    
    # return the taxon dataframe
    return df_taxa

def prepare_plotting_dfs(df, file_dict, sortby=list(file_dict.keys())):
    # transpose dataframe, sort counts, and add new column names
    df_transpose = df.sort_values(by=sortby, ascending=False).set_index('Taxa').transpose().rename_axis('Samples', axis=1)
    # normalize the species counts of each sample 100%
    df_transpose_norm = df_transpose.apply(lambda x: x*100/sum(x), axis=1)
    
    return df_transpose, df_transpose_norm

def get_all_dfs(file_dict, rank, sortby=list(file_dict.keys())):
    df_one = make_rank_dataframe(file_dict, rank)
    df_two, df_three = prepare_plotting_dfs(df_one, file_dict, sortby=sortby)
    return df_one, df_two, df_three
    

In [ ]:
########################################################################
# Create a taxon dataframe for the SPECIES level
########################################################################

# This demonstrates how to use the main function get_all_dfs() to create 
# the comparison dataframes. We will start the demonstration for the 
# species level.

# To generate the dataframes, we use the following function: 
# get_all_dfs(file_dict, rank, sortby='Name')
#    the first positional argument is file_dict (leave unchanged)
#    the second positional argument is the rank desired ('S1', 'S', 'G', 'F', etc.)
#    the optional argument sortby can be used to specify one of the samples to sort 
#        abundances by, which must be a key name in file_dict
# The function returns three dataframes, so three variable names must be specified

# Here is the basic usage to get dataframes for species level taxa:
df_sp, df_sp_plot, df_sp_plot_norm = get_all_dfs(file_dict, 'S')
# In the above example, we labeled the three dataframes returned by the function as:
# df_sp, df_sp_plot, and df_sp_plot_norm.

# Below is the same example but with sorting by abundances in sample "CID_139445", un-hash to run:
# df_sp, df_sp_plot, df_sp_plot_norm = get_all_dfs(file_dict, 'S', sortby="CID_139445")
# Make sure if you use the optional argument that you specify a label name/key in the file_dict.

In [ ]:
# Preview the first type of dataframe. Here, samples are the columns and taxa form the rows. 
df_sp.head()

In [ ]:
# Preview the second dataframe. Notice it is a transposed version of the first one.
# Here, taxa are columns and samples are the rows.
df_sp_plot.head()

In [ ]:
# Preview the third dataframe. This is the same format as above,
# but the values have been normalized so counts for each sample 
# total to 100. 
df_sp_plot_norm.head()

In [ ]:
########################################################################
# Saving the dataframes as CSV files
########################################################################

# to save any of the above dataframes, use the df.to_csv() method

# for the df_sp, we don't want to include the index (which is just integers)
# we can also specify tab-delimited using the sep argument:
df_sp.to_csv("Species-Absolute-Abundance-Counts.txt", index=False, sep='\t')

# for the df_sp_plot and df_sp_plot_norm dataframes, keep the index (which is the sample names):
df_sp_plot.to_csv("Species-Absolute-Abundance-Counts-Transposed.txt", sep='\t')


In [ ]:
########################################################################
# Make a HORIZONTAL stacked barplot using the NORMALIZED COUNT dataframe.
########################################################################

# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# To make plotting easier we can set the color palette here using seaborn. 
pal = sns.color_palette("tab20")

# Create the horizontal stacked bar plot using the pandas .plot.barh() function with the following args:

# stacked: controls whether or not to create a stacked barplot
# figsize: the dimensions of the figure, in (width, height)
# width: this controls the width of the bars, 1 = no separation between them, <1 puts spacing in
# color: sets the colors, we refer to pal which was set above
# fontsize: the size of sample labels (values = xx-small, x-small, small, medium, large, x-large, xx-large)
# edgecolor: controls the color of lines separating colors within bars, as well as bar outline
# linewidth: the line size for the edgecolor

ax = df_sp_plot_norm.plot.barh(stacked=True, figsize=(20,5), width=0.8, 
                               color=pal, fontsize='large', edgecolor='black', 
                               linewidth=0.5)

# these commands eliminate the bounding box for the barplot
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)

# this controls the placement of the legend (bbox_to_anchor), as well as font (fontsize), 
# number of columns for labels (ncol), and spacing (labelspacing)
ax.legend(bbox_to_anchor=(1,-0.10), fontsize='x-small', ncol=7, labelspacing=0.3)

In [ ]:
# You can save the above plot using the following:
ax.figure.savefig('Species-Abundances-Normalized-Horizontal.pdf', bbox_inches='tight')

In [ ]:
########################################################################
# Make a VERTICAL stacked barplot using the NORMALIZED COUNT dataframe.
########################################################################

# Here you will need to potentially change some of the argument values 
# depending on the number of samples and taxa in your dataset.

# To make plotting easier we can set the color palette here using seaborn. 
pal = sns.color_palette("tab20")

# Create the vertical stacked bar plot using the pandas .plot.bar() function with the following args:

# stacked: controls whether or not to create a stacked barplot
# figsize: the dimensions of the figure, in (width, height)
# width: this controls the width of the bars, 1 = no separation between them, <1 puts spacing in
# color: sets the colors, we refer to pal which was set above
# fontsize: the size of sample labels (values = xx-small, x-small, small, medium, large, x-large, xx-large)
# edgecolor: controls the color of lines separating colors within bars, as well as bar outline
# linewidth: the line size for the edgecolor

ax = df_sp_plot_norm.plot.bar(stacked=True, figsize=(5,15), width=0.8, 
                               color=pal, fontsize='large', edgecolor='black', 
                               linewidth=0.5)

# these commands eliminate the bounding box for the barplot
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# for vertical plot, we need to reverse the legend labels, start by getting these objects
handles, labels = ax.get_legend_handles_labels()

# and add reversed(handles) and reversed(labels) to the legend arguments
# this controls the placement of the legend (bbox_to_anchor), as well as font (fontsize), 
# number of columns for labels (ncol), and spacing (labelspacing)
ax.legend(reversed(handles), reversed(labels), bbox_to_anchor=(1,0.9), 
          fontsize='x-small', ncol=1, labelspacing=0.3)

In [ ]:
# You can save the above plot using the following:
ax.figure.savefig('Species-Abundances-Normalized-Vertical.pdf', bbox_inches='tight')

In [ ]:
########################################################################
# Make a HORIZONTAL stacked barplot using the ABSOLUTE COUNT dataframe.
########################################################################

# In this plot, we don't expect the stacked bars to be the same lengths.

# All the same arguments are used, but we use the df_sp_plot dataframe instead
# of the df_sp_plot_norm dataframe

pal = sns.color_palette("tab20")
ax = df_sp_plot.plot.barh(stacked=True, figsize=(20,5), width=0.8, 
                               color=pal, fontsize='large', edgecolor='black', 
                               linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(bbox_to_anchor=(1,-0.10), fontsize='x-small', ncol=7, labelspacing=0.3)

In [ ]:
# You can save the above plot using the following:
ax.figure.savefig('Species-Abundances-Absolute-Horizontal.pdf', bbox_inches='tight')

In [ ]:
########################################################################
# Create a taxon dataframe for the GENUS level
########################################################################

# Do the same thing, but for the genus level

# here is the basic usage to get dataframes for genus level taxa
df_gn, df_gn_plot, df_gn_plot_norm = get_all_dfs(file_dict, 'G')


In [ ]:
# Preview the first dataframe. Here, samples are the columns and taxa form the rows.
df_gn.head()

In [ ]:
# Save genus counts to csv
df_gn.to_csv("Genus-Absolute-Abundance-Counts.txt", index=False, sep='\t')

In [ ]:
########################################################################
# Make a HORIZONTAL stacked barplot using the NORMALIZED COUNT dataframe.
########################################################################

# make sure to change dataframe name to plot!

pal = sns.color_palette("tab20")

ax = df_gn_plot_norm.plot.barh(stacked=True, figsize=(20,5), width=0.8, 
                               color=pal, fontsize='large', edgecolor='black', 
                               linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(bbox_to_anchor=(0.9,-0.10), fontsize='small', ncol=8, labelspacing=0.3)

In [ ]:
########################################################################
# Create a taxon dataframe for the FAMILY level
########################################################################

# Do the same thing, but for the family level

# here is the basic usage to get dataframes for genus level taxa
df_fm, df_fm_plot, df_fm_plot_norm = get_all_dfs(file_dict, 'F')


In [ ]:
# Preview the first dataframe. Here, samples are the columns and taxa form the rows.
df_fm.head()

In [ ]:
# Save family counts to csv
df_fm.to_csv("Family-Absolute-Abundance-Counts.txt", index=False, sep='\t')

In [ ]:
########################################################################
# Make a HORIZONTAL stacked barplot using the NORMALIZED COUNT dataframe.
########################################################################

# make sure to change dataframe name to plot!

pal = sns.color_palette("tab20")

ax = df_fm_plot_norm.plot.barh(stacked=True, figsize=(20,5), width=0.8, 
                               color=pal, fontsize='large', edgecolor='black', 
                               linewidth=0.5)
ax.spines['top'].set_visible(False)
ax.spines['left'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.legend(bbox_to_anchor=(0.8,-0.10), fontsize='small', ncol=7, labelspacing=0.3)